In [28]:
from qiskit.opflow import I, X, Y, Z
from qiskit.circuit import ParameterVector, QuantumCircuit
from qiskit import Aer
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.opflow import CX, T, H, S, PrimitiveOp, CircuitOp
from qiskit.opflow import StateFn, CircuitStateFn
import numpy as np
H_op = (0.5 * Z)

ansatz = QuantumCircuit(1)
nu = ParameterVector('nu', 2)
ansatz.h(0)
ansatz.rz(nu[0], 0)
ansatz.ry(nu[1], 0)

opt = SLSQP(maxiter=1000)
qi = QuantumInstance(Aer.get_backend('statevector_simulator'))
vqe = VQE(ansatz, optimizer=opt, quantum_instance=qi)
result = vqe.compute_minimum_eigenvalue(operator=H_op)

opt_circ = vqe.get_optimal_circuit() #depricated
test_state = CircuitOp(opt_circ)
zero_proj = (0.5 * (I + Z))
#
# H_top = H_op + test_state.adjoint() @ zero_proj @ test_state
# this line doesn't work because we are having 
# CircuitOp @ PauliOp @ CircuitOp which is a mix
# therefore let's try Kyle's suggestion
measure_op = StateFn(H_op, is_measurement=True) @ CircuitStateFn(ansatz)
overlap_op = StateFn(zero_proj, is_measurement=True) @ \
        CircuitStateFn(ansatz + opt_circ.inverse())
print(measure_op)
print(overlap_op)
H_top = measure_op + overlap_op
print(H_top)
H_top = (I + Z) / np.sqrt(2)
print(H_top)
result_ex = vqe.compute_minimum_eigenvalue(operator=H_top)
print(result_ex)

ComposedOp([
  OperatorMeasurement(0.5 * Z),
  CircuitStateFn(
       ┌───┐┌───────────┐┌───────────┐
  q_0: ┤ H ├┤ Rz(nu[0]) ├┤ Ry(nu[1]) ├
       └───┘└───────────┘└───────────┘
  )
])
ComposedOp([
  OperatorMeasurement(0.5 * I
  + 0.5 * Z),
  CircuitStateFn(
       ┌───┐┌───────────┐┌───────────┐┌──────────────────────┐»
  q_0: ┤ H ├┤ Rz(nu[0]) ├┤ Ry(nu[1]) ├┤ Ry(1.57078627758347) ├»
       └───┘└───────────┘└───────────┘└──────────────────────┘»
  «     ┌──────────────────────┐┌───┐
  «q_0: ┤ Rz(3.14171430611855) ├┤ H ├
  «     └──────────────────────┘└───┘
  )
])
SummedOp([
  ComposedOp([
    OperatorMeasurement(0.5 * Z),
    CircuitStateFn(
         ┌───┐┌───────────┐┌───────────┐
    q_0: ┤ H ├┤ Rz(nu[0]) ├┤ Ry(nu[1]) ├
         └───┘└───────────┘└───────────┘
    )
  ]),
  ComposedOp([
    OperatorMeasurement(0.5 * I
    + 0.5 * Z),
    CircuitStateFn(
         ┌───┐┌───────────┐┌───────────┐┌──────────────────────┐»
    q_0: ┤ H ├┤ Rz(nu[0]) ├┤ Ry(nu[1]) ├┤ Ry(1.57078627758347